In [24]:
result = {'barcode': '2500000279935', 'cert_mark': ['organic'], 'cert_result': {'12100489': {'in_db': True, 'mark_status': 'success', 'name': {'db': '김영대', 'ocr_rslt': '김영대', 'score': 1.0}, 'number': {'db': '12100489', 'ocr_rslt': '12100409', 'score': 0.875}}}, 'date_time': '02/15/2022, 15:35:24', 'label_id': 157, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000279935/0157.png', 'product_name': {'name': '친환경 방울토마토', 'status': 'success'}, 'rot_angle': 180, 'weight': {'db': '600g', 'ocr_rslt': '600g', 'score': 1.0, 'status': 'success'}}

result1 = {'barcode': '2500000279935', 'cert_mark': ['organic'], 'cert_result': {'04829818': {'in_db': False, 'mark_status': 'success', 'number': {'db': '04829818', 'ocr_rslt': '04829818'}}, '12100489': {'in_db': True, 'mark_status': 'success', 'name': {'db': '김영대', 'ocr_rslt': '김영대', 'score': 1.0}, 'number': {'db': '12100489', 'ocr_rslt': '12100489', 'score': 1.0}}}, 'date_time': '02/15/2022, 16:40:53', 'label_id': 562, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000279935/0562.png', 'product_name': {'name': '친환경 방울토마토', 'status': 'success'}, 'rot_angle': 0, 'weight': {'db': '600g', 'ocr_rslt': '600g', 'score': 1.0, 'status': 'success'}}

result2 = {'barcode': '2500000279935', 'cert_mark': ['organic'], 'cert_result': {}, 'date_time': '02/16/2022, 17:00:02', 'label_id': 1405, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000279935/1405.png', 'product_name': {'name': '친환경 방울토마토', 'status': 'success'}, 'rot_angle': 180, 'weight': {'db': '600g', 'ocr_rslt': '600g', 'score': 1.0, 'status': 'success'}}

mark_kor = {
    'organic': '유기농',
    'nonpesticide': '무농약',
    'gap': 'GAP(우수관리인증)',
    'antibiotic': '무항생제',
    'animal': '동물복지',
    'haccp': '안전관리인증HACCP',
    'pgi': '지리적표시',
    'traditional': '전통식품',
    'master': '식품명인',
    'processed': '가공식품',
    'carbon': '저탄소(LOW CARBON)'
}

def inspection_result(result):
    product_name = []
    weight = []
    cert_marks = []
    cert_results = []
    barcode = []

    for key, value in result.items():
        cert_third_num = []
        if key == 'product_name':
            product_name.append(value['name'])
            product_name.append(value['name'])
            if value['status'] == 'success':
                product_name.append('100%')
                product_name.append('승인')

        if key == 'weight':
            weight.append(value['ocr_rslt'])
            weight.append(value['db'])
            weight.append(f"{round(value['score']*100)}%")
            weight.append('승인')

        if key == 'barcode':
            barcode.append(value)
            barcode.append(value)
            barcode.append('100%')
            barcode.append('승인')

        if key == 'cert_mark':
            for mark in value:
                mark_lst = []
                mark_lst.append(mark_kor[mark])
                cert_numbers = [item['number']['ocr_rslt'] for item in result['cert_result'].values()]
                mark_lst.append(', '.join([number[2] for number in cert_numbers]))
                mark_status = [number['mark_status'] for number in result['cert_result'].values()]
                count = 0
                for status in mark_status:
                    if 'success' == status:
                        count += 1
                raw_score = 0.0
                if len(mark_status) > 0:
                    raw_score = count / len(mark_status)

                score = f"{round(raw_score*100)}%"
                mark_lst.append(score)
                if raw_score >= 0.9:
                    mark_lst.append('승인')
                cert_marks.append(mark_lst)

        if key == 'cert_result':
            for num_key, num_value in value.items():
                cert_lst = []
                if num_value['in_db']:
                    cert_lst.append(f"{num_value['number']['ocr_rslt']} {num_value['name']['ocr_rslt']}")
                    cert_lst.append(f"{num_value['number']['db']} {num_value['name']['db']}")
                    raw_score = (num_value['name']['score'] + num_value['number']['score']) / 2.0
                    cert_lst.append(f"{round(raw_score)*100}%")
                    result = '오류'
                    if raw_score >= 0.9:
                        result = '승인'
                    cert_lst.append(result)

                else:
                    name = ''
                    number = ''
                    if 'name' in num_value and 'number' in num_value:
                        name = num_value['name']['ocr_rslt']
                        number = num_value['number']['ocr_rslt']
                    elif 'number' in num_value.keys():
                        number = num_value['number']['ocr_rslt']
                    elif 'name' in num_value.keys():
                        name = num_value['name']['ocr_rslt']
                    cert_lst.append(f"{name}, {number}")
                    cert_lst.append("매칭실패")
                    cert_lst.append('0%')
                    cert_lst.append('매칭실패')
                cert_results.append(cert_lst)
                # cert_results.append(info)

    return {'product_name': product_name, 'weight': weight, 'barcode': barcode, 'cert_marks': cert_marks, 'cert_results': cert_results}

In [25]:
result_dict = inspection_result(result1)
result_dict

{'product_name': ['친환경 방울토마토', '친환경 방울토마토', '100%', '승인'],
 'weight': ['600g', '600g', '100%', '승인'],
 'barcode': ['2500000279935', '2500000279935', '100%', '승인'],
 'cert_marks': [['유기농', '8, 1', '100%', '승인']],
 'cert_results': [[', 04829818', '매칭실패', '0%', '매칭실패'],
  ['12100489 김영대', '12100489 김영대', '100%', '승인']]}